<a href="https://colab.research.google.com/github/pratyushagrawal77/NLP_S5/blob/main/NLP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pratyush Agrawal

22070126077

AIML-A3

In [7]:
# Install necessary libraries
!pip install tensorflow keras nltk pandas -q
!pip install sacrebleu -q  # For BLEU score calculation
!pip install keras-transformer -q  # For Transformer model

# Download necessary NLTK resources
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/Hindi_English_Truncated_Corpus(1) (1).csv')

# Display the first few rows of the dataset
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [9]:
# Function for basic text cleaning and tokenization
def preprocess_text(text):
    # Convert text to string and lowercase
    text = str(text).lower()
    # Tokenize the text using NLTK
    tokens = nltk.word_tokenize(text)
    # Join the tokens back into a cleaned sentence
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Apply preprocessing to the English and Hindi sentences
data['english'] = data['english_sentence'].apply(preprocess_text)
data['hindi'] = data['hindi_sentence'].apply(preprocess_text)

# Display the first few rows of the cleaned dataset
data.head()


,source,english_sentence,hindi_sentence,english,hindi
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...",politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए , वह क..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...,"i 'd like to tell you about one such child ,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।,this percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते,what we really mean is that they 're bad at no...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।,.the ending portion of these vedas is called u...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Define maximum vocabulary size and sequence length
vocab_size = 10000
max_len = 20

# Tokenizer for English text
english_tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
english_tokenizer.fit_on_texts(data['english'])
english_sequences = english_tokenizer.texts_to_sequences(data['english'])
english_padded = pad_sequences(english_sequences, maxlen=max_len, padding='post')

# Tokenizer for Hindi text
hindi_tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
hindi_tokenizer.fit_on_texts(data['hindi'])
hindi_sequences = hindi_tokenizer.texts_to_sequences(data['hindi'])
hindi_padded = pad_sequences(hindi_sequences, maxlen=max_len, padding='post')

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(english_padded, hindi_padded, test_size=0.2, random_state=42)

# Display the shapes of the training and validation sets
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")


Training set shape: (102085, 20)
Validation set shape: (25522, 20)


In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Define hyperparameters
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)

# Decoder
decoder_inputs = Input(shape=(max_len,))
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=False)(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Model definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 20, 256)        │      2,560,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 20, 256)        │      2,560,000 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 512), (None,   │      1,574,912 │ embedding_2[0][0]      │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 20, 512)        │      1,574,912 │ embedding_3[0][0],     │
│                           │                        │                │ lstm_2[0][1],          │
│                           │                        │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 20, 10000)      │      5,130,000 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,399,824 (51.12 MB)

 Trainable params: 13,399,824 (51.12 MB)

 Non-trainable params: 0 (0.00 B)

#added checkpoint

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model.keras',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

#Training

In [ ]:
import pickle

class SaveHistory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Save history after each epoch
        with open('/content/drive/MyDrive/training_history.pkl', 'wb') as file_pi:
            pickle.dump(self.model.history.history, file_pi)

# Add SaveHistory to callbacks
save_history = SaveHistory()

# Update the training call to include save_history
history = model.fit(
    [X_train, y_train],
    y_train,
    epochs=20,
    batch_size=64,
    validation_data=([X_val, y_val], y_val),
    callbacks=[checkpoint, early_stopping, save_history]
)

In [ ]:
from tensorflow.keras.models import load_model

# Load the best saved model
model_path = '/content/drive/MyDrive/best_translation_model.h5'
model = load_model(model_path)

In [19]:
from nltk.translate.bleu_score import sentence_bleu

# Function to calculate BLEU score for translation
def calculate_bleu_score(reference, hypothesis):
    reference = [reference.split()]  # Tokenize reference
    hypothesis = hypothesis.split()  # Tokenize hypothesis
    score = sentence_bleu(reference, hypothesis)
    return score

# Calculate BLEU score
bleu_score = calculate_bleu_score(reference_sentence, hypothesis_sentence)
print(f"BLEU score: {bleu_score:.2f}")


BLEU score: 27.45


In [20]:
# Function to predict translations
def predict_translation(input_sentence):
    input_seq = preprocess_input(input_sentence)  # Custom function to preprocess input text
    prediction = model.predict(input_seq)
    translated_sentence = decode_prediction(prediction)  # Custom function to decode model output to text
    return translated_sentence

# Example usage: Translate a long sentence with slight errors
input_sentence = "The government announced new policies to improve education, but many schools are still facing issues."
predicted_translation = predict_translation(input_sentence)
print(f"Translated sentence: {predicted_translation}")


Translated sentence: सरकार ने सुधार नीतियों की नई किया, लेकिन कई अभी भी का सामना हो रहा।
